# Тетрадка 5. RL Инструментарий

## Содержание:

1. [Модификация сред: Wrappers](#wrappers)
2. [Логирование Wandb](#logs)
3. [RL Фреймворки](#frameworks)


In [ ]:
# @title Установка зависимостей

try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [ ]:
# @title Импортирование зависимостей

import time
import glob
import io
import base64
from IPython import display as ipythondisplay
from IPython.display import HTML
import gymnasium as gym
from gymnasium.wrappers.record_video import RecordVideo
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


def show_video(folder="./video"):
    mp4list = glob.glob(folder + '/*.mp4')
    if len(mp4list) > 0:
        mp4 = sorted(mp4list, key=lambda x: x[-15:], reverse=True)[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

# 1. Модификация сред: Wrappers  <a name = 'wrappers'></a>
Мы знаем, что окружения создаются с помощью команды ``gym.make(<имя среды>)``, но что если мы хотим немного изменить окружение или добавить какой-то дополнительный функционал? Для этого существуют обертки (wrappers). [Уже существующие обертки](https://gymnasium.farama.org/api/wrappers/).


## Reward Wrapper
 Вспомним среду ``Taxi-v3``, предположим, что мы хотим поменять вознаграждения на следующие: 1 за решение задачи, -1 за неправильную посадку/высадку пассажира и 0 во всех остальных случаях. Для этого мы можем воспользоваться ``RewardWrapper``-ом:

In [ ]:
class MyRewardWrapper(gym.RewardWrapper):
    def reward(self, reward):
        if reward == -1:
            return 0
        elif reward == 20:
            return 1
        elif reward == -10:
            return -1
        else:
            raise KeyError

In [ ]:
env = gym.make("Taxi-v3")
env = MyRewardWrapper(env)
observation, info = env.reset()

rewards = set()

while True:
    observation, reward, term, trunc, info = env.step(env.action_space.sample())
    rewards.add(reward)
    if term:
        break

# выведем все вознаграждения, которые получал агент
print(rewards)
env.close()

{0, 1, -1}


## Time Limit Wrapper

В зависимости от случая, мы могли получить разные результаты, но обычно это ``{0, -1}``. Откуда такой результат? Ведь среда заканчивается только когда задание выполнено. Все дело во встроенной обертке ограничивающей максимальное количество шагов.

In [ ]:
env = gym.make("Taxi-v3")
print(type(env))
env.close()

<class 'gymnasium.wrappers.time_limit.TimeLimit'>


Можно воспользоваться окружением без этой обертки, вызвав ``.env``:

In [ ]:
env = gym.make("Taxi-v3", max_episode_steps=2)
env = MyRewardWrapper(env)

observation, info = env.reset()
rewards = set()
while True:
    observation, reward, term, trunc, info = env.step(env.action_space.sample())
    rewards.add(reward)
    if term or trunc:
        break

print(rewards)
env.close()

Step
Step
{0}


In [ ]:
term, trunc

(True, False)

А если у нас есть окружение без этой обертки по умолчанию, то можно добавить его вот так:

In [ ]:
env = gym.make("Taxi-v3").env
env = MyRewardWrapper(env)
env = gym.wrappers.TimeLimit(env, max_episode_steps=1)

observation, info = env.reset()
rewards = set()

while True:
    observation, reward, term, trunc, info = env.step(env.action_space.sample())
    rewards.add(reward)
    if term or trunc:
        break

print(rewards)
env.close()

step
{0}


In [ ]:
term, trunc

Как вы считаете, корректно ли с точки зрения MDP, если среда будет досрочно возвращать ``done=True``?

## Action Wrapper
Представим, что наш водитель находится не в лучшем своем состоянии и независимо от выбора агента, в 50% случаев совершает случайные действий. Сделать среду стохастической и добиться такого эффекта мы можем, используя ``ActionWrapper``:

In [ ]:
class TaxiRandomActionWrapper(gym.ActionWrapper):
    def __init__(self, env, probability=0.5):
        super().__init__(env)
        self.probability = probability

    def action(self, action):
        if np.random.random() < self.probability:
            return env.action_space.sample()
        else:
            return action

Чтобы проверить, что обертка работает будем выполнять единственное действие:

In [ ]:
env = gym.make("Taxi-v3", render_mode='rgb_array')
env = MyRewardWrapper(env)
env = TaxiRandomActionWrapper(env)
env = RecordVideo(env, f"./video")

observation, info = env.reset()
rewards = set()

while True:
    action = 0
    observation, reward, term, trunc, info = env.step(action)
    rewards.add(reward)
    if term or trunc:
        break

print(rewards)
env.close()
show_video()

/usr/local/lib/python3.10/dist-packages/gymnasium/wrappers/record_video.py:87: UserWarning: WARN: Overwriting existing videos at /content/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /content/video/rl-video-episode-0.mp4.
Moviepy - Writing video /content/video/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/video/rl-video-episode-0.mp4
{0, -1}


## Wrapper

Класс ``gym.Wrapper`` является базовым для всех оберток. Подкласс может переопределить    многие методы для изменения поведения исходной среды, не изменяя при этом ее исходный код. Например, мы можем изменить метод step и добавить, какую-то дополнительную информацию в ``info``:

In [ ]:
class MyWrapper(gym.Wrapper):
    def step(self, action):
        observation, reward, term, trunc, info = self.env.step(action)
        info['Wrapped'] = True
        return observation, reward, term, trunc, info

In [ ]:
env = gym.make("Taxi-v3")
env = MyWrapper(env)

env.reset()
_, _, _, _, info = env.step(env.action_space.sample())

print(info)
env.close()

{'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8), 'Wrapped': True}


## Atari Preprocessing


Предварительная обработка для Atari 2600.
Класс ``AtariPreprocessing`` следует рекомендациям статьи: Revisiting the Arcade Learning Environment: Evaluation Protocols and Open Problems for General Agents" Machado et al. (2018).

In [ ]:
env = gym.make("BoxingNoFrameskip-v0")

print(f'Original: {env.observation_space.shape}')

Original: (210, 160, 3)


In [ ]:
env = gym.wrappers.AtariPreprocessing(gym.make("BoxingNoFrameskip-v0"))

print(f'Preprocessed: {env.observation_space.shape}')

Preprocessed: (84, 84)


# 2. Логирование Wandb <a name = 'logs'></a>

Несколько примеров: [Dexterity](https://wandb.ai/site/customers/learning-dexterity-end-to-end-using-weights-biases-reports), [RL Example: Pacman](https://wandb.ai/yashkotadia/rl-example).

Для того, чтобы результаты записывались в облако, нужно залогиниться. Предоставить свой api-key. Самый простой способ - использование консоли:

In [ ]:
if COLAB:
    !pip install wandb --quiet

import wandb
wandb.login()

ERROR: Operation cancelled by user


wandb: Currently logged in as: arkol. Use `wandb login --relogin` to force relogin


True

In [ ]:
import math
import random

# Start a new run, tracking hyperparameters in config
run = wandb.init(project="test-drive", config={
    "learning_rate": 0.01,
    "dropout": 0.2,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
})
config = run.config

# Simulating a training or evaluation loop
for x in range(50):
    acc = math.log(1 + x + random.random()*config.learning_rate) + random.random() + config.dropout
    loss = 10 - math.log(1 + x + random.random() + config.learning_rate*x) + random.random() + config.dropout
    # Log metrics from your script to W&B
    run.log({"acc":acc, "loss":loss}, step=x)
run.finish()

acc,▁▂▂▄▄▄▅▅▄▄▅▅▅▅▆▆▆▆▆▅▇▇▆▇▇▆▆▆▆▆▆▇▆▇▇▇▆███
loss,██▇▆▆▆▆▄▄▅▅▄▄▃▃▃▃▃▃▄▄▃▃▂▂▃▂▃▁▃▃▂▃▁▂▁▁▁▂▁
acc,5.02756
loss,6.58174


# 3. RL Фреймворки <a name = 'frameworks'></a>

Переиспользование написанного кода является хорошей практикой написания программ и обучение с подкреплением не является исключением. Существует большое число готовых реализаций RL алгоритмов. Рассмотрим некоторые из них:

## Stable Baselines3


Stable Baselines3 (SB3) is a set of reliable implementations of reinforcement learning algorithms in PyTorch. It is the next major version of [Stable Baselines](https://github.com/DLR-RM/stable-baselines3).

You can read a detailed presentation of Stable Baselines3 in the [v1.0 blog post](https://araffin.github.io/post/sb3/) or our [JMLR paper](https://jmlr.org/papers/volume22/20-1364/20-1364.pdf).


These algorithms will make it easier for the research community and industry to replicate, refine, and identify new ideas, and will create good baselines to build projects on top of. We expect these tools will be used as a base around which new ideas can be added, and as a tool for comparing a new approach against existing ones. We also hope that the simplicity of these tools will allow beginners to experiment with a more advanced toolset, without being buried in implementation details.


**Neural Network Framework:** PyTorch.

**Список алгоритмов:** A2C, DDPG, DQN, HER, PPO, SAC, TD3, etc.

Tutorial: [Stable Baselines3 Tutorial](https://github.com/araffin/rl-tutorial-jnrr19)

Getting started: [Colab](https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/sb3/1_getting_started.ipynb)

## CleanRL

[CleanRL](https://docs.cleanrl.dev/) is a Deep Reinforcement Learning library that provides high-quality single-file implementation with research-friendly features. The implementation is clean and simple, yet we can scale it to run thousands of experiments using AWS Batch.


## PureJaxRL
[PureJaxRL](https://github.com/luchris429/purejaxrl) is a high-performance, end-to-end Jax Reinforcement Learning (RL) implementation. When running many agents in parallel on GPUs, our implementation is over 1000x faster than standard PyTorch RL implementations. Unlike other Jax RL implementations, we implement the entire training pipeline in JAX, including the environment. This allows us to get significant speedups through JIT compilation and by avoiding CPU-GPU data transfer. It also results in easier debugging because the system is fully synchronous. More importantly, this code allows you to use jax to jit, vmap, pmap, and scan entire RL training pipelines.

## Rllib

[RLlib](https://docs.ray.io/en/master/rllib.html) $-$ это open source RL фреймворк, который предлагает высокую масштабируемость и унифицированный API. RLlib изначально поддерживает TensorFlow, TensorFlow Eager и PyTorch. Для распараллеливания используется [Ray Project](https://github.com/ray-project/ray).

**Neural Network Framework:** Tensorflow, PyTorch.

**[Список алгоритмов:](https://docs.ray.io/en/master/rllib-algorithms.html)** A2C, A3C, ARS, BC, ES, DDPG, TD3, APEX-DDPG, Dreamer, DQN, Rainbow, APEX-DQN, IMPALA, MAML, MARWIL, MBMPO, PG, PPO, APPO, SAC, LinUCB, LinTS, AlphaZero, QMIX, MADDPG


### PFRL


PFRL $-$ это библиотека глубокого обучения с подкреплением, которая реализует различные современные алгоритмы на Python, с использованием PyTorch. Бывший ChainerRL.

**Neural Network Framework:** PyTorch.

**Список алгоритмов:** DQN, Rainbow, IQN, DDPG, A3C, ACER, PPO, TRPO, TD3, SAC

Быстрый старт: [quickstart.ipynb](https://github.com/pfnet/pfrl/blob/master/examples/quickstart/quickstart.ipynb)

Примеры: [examples](https://github.com/pfnet/pfrl/tree/master/examples)

### Заслуживают упоминания:

* [OpenAI Spinning Up RL](https://github.com/openai/spinningup)
* [LeelaChessZero](https://github.com/LeelaChessZero)
* [Tianshou](https://github.com/thu-ml/tianshou)
* [TF agents](https://github.com/tensorflow/agents)
* [Catalyst-rl](https://github.com/catalyst-team/catalyst-rl)
* [Dopamine](https://github.com/google/dopamine)
* [TRFL](https://github.com/deepmind/trfl)
* [Keras RL](https://github.com/keras-rl/keras-rl)
* [PyTorch-RL](https://github.com/Khrylx/PyTorch-RL)
* [PyMarl](https://github.com/oxwhirl/pymarl)
* [Denny Britz RL](https://github.com/dennybritz/reinforcement-learning)
* [DeepRL-Tutorials](https://github.com/qfettes/DeepRL-Tutorials)
* [RL Adventure](https://github.com/higgsfield/RL-Adventure)
* [FacebookResearch ELF](https://github.com/facebookresearch/ELF)
* [SLM-Lab](https://github.com/kengz/SLM-Lab)
